In [92]:
import pandas as pd
import numpy as np
import requests
from scipy.stats import t
import plotly.express as px

In [93]:
odds = pd.read_csv("/media/hdd/tmp/thoroughbred-parsed/thoroughbred-odds-2021.csv", dtype={'market_id': object, 'selection_id': object})

race = pd.read_csv("/media/hdd/tmp/thoroughbred-parsed/thoroughbred-race-data.csv", dtype={'market_id': object, 'selection_id': object})


/home/tmbish/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning:

Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.

/home/tmbish/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning:

Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.



In [94]:
odds.head(5)

,market_id,selection_id,selection_name,wap_5m,wap_30s,bsp,ltp,traded_vol,inplay_min_lay
0,1.179845158,23493550,1. Larmour,6.27,5.84,6.20,6.2,8277,1.19
1,1.179845158,16374800,3. Careering Away,3.31,3.67,3.60,3.65,18592,1.08
2,1.179845158,19740699,4. Bells N Bows,6.87,6.36,6.62,6.4,7413,1.42
3,1.179845158,11014075,5. Tickinover,40.73,42.75,54.42,50.0,725,24.00
4,1.179845158,21317306,6. Opera Tickets,21.54,22.38,22.22,21.0,1972,18.50


In [95]:
race.head(5)

,date,track,country,race_type,race_number,market_id,start_time,weather,track_condition,race_distance,selection_id,selection_name,barrier,place,trainer,jockey,weight
0,2020-07-01,Balaklava,AUS,R,1,1.171091087,2020-07-01 02:40:00,FINE,GOOD4,2200,19674744,Baldy,2,4.0,Peter Nolan,Karl Zechner,59.0
1,2020-07-01,Balaklava,AUS,R,1,1.171091087,2020-07-01 02:40:00,FINE,GOOD4,2200,401615,Nostrovia,4,7.0,Dennis O'leary,Margaret Collett,59.0
2,2020-07-01,Balaklava,AUS,R,1,1.171091087,2020-07-01 02:40:00,FINE,GOOD4,2200,26789410,Ammo Loco,5,1.0,John Hickmott,Barend Vorster,58.5
3,2020-07-01,Balaklava,AUS,R,1,1.171091087,2020-07-01 02:40:00,FINE,GOOD4,2200,26623561,Gunfireinthecity,7,3.0,Dennis O'leary,Kayla Crowther,57.0
4,2020-07-01,Balaklava,AUS,R,1,1.171091087,2020-07-01 02:40:00,FINE,GOOD4,2200,28699693,Whys,3,2.0,Trevor Day,Sairyn Fawke,57.0


In [96]:
# Joining two datasets

df = race.merge(odds.loc[:, odds.columns != 'selection_name'], how = "inner", on = ['market_id', 'selection_id'])

#df = df[['date', 'market_id', 'selection_id', 'country', 'track', 'race_type', 'race_number', 'start_time', 'weather', 'track_condition', 'race_distance', 'barrier', '']]
df

,date,track,country,race_type,race_number,market_id,start_time,weather,track_condition,race_distance,...,place,trainer,jockey,weight,wap_5m,wap_30s,bsp,ltp,traded_vol,inplay_min_lay
0,2020-07-01,Balaklava,AUS,R,1,1.171091087,2020-07-01 02:40:00,FINE,GOOD4,2200,...,4.0,Peter Nolan,Karl Zechner,59.0,3.30,3.76,3.49,3.5,16543,2.70
1,2020-07-01,Balaklava,AUS,R,1,1.171091087,2020-07-01 02:40:00,FINE,GOOD4,2200,...,7.0,Dennis O'leary,Margaret Collett,59.0,45.00,72.01,102.90,100.0,583,130.00
2,2020-07-01,Balaklava,AUS,R,1,1.171091087,2020-07-01 02:40:00,FINE,GOOD4,2200,...,1.0,John Hickmott,Barend Vorster,58.5,2.80,2.46,2.74,3.0,36043,1.01
3,2020-07-01,Balaklava,AUS,R,1,1.171091087,2020-07-01 02:40:00,FINE,GOOD4,2200,...,3.0,Dennis O'leary,Kayla Crowther,57.0,14.41,11.25,14.00,14.0,4113,7.40
4,2020-07-01,Balaklava,AUS,R,1,1.171091087,2020-07-01 02:40:00,FINE,GOOD4,2200,...,2.0,Trevor Day,Sairyn Fawke,57.0,11.37,13.58,13.62,13.5,2395,13.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160022,2021-06-29,Townsville,AUS,R,6,1.184845396,2021-06-29 07:00:00,None,None,1000,...,1.0,Sharlee D'avila,Lacey Morrison,57.5,7.22,6.48,5.97,5.9,14848,1.01
160023,2021-06-29,Townsville,AUS,R,6,1.184845396,2021-06-29 07:00:00,None,None,1000,...,7.0,Graham Hughes,Frank Edwards,56.0,67.74,50.79,70.42,75.0,677,75.00
160024,2021-06-29,Townsville,AUS,R,6,1.184845396,2021-06-29 07:00:00,None,None,1000,...,4.0,Alby Molino,M Potgieter,55.0,12.55,11.02,13.24,14.0,6700,10.00
160025,2021-06-29,Townsville,AUS,R,6,1.184845396,2021-06-29 07:00:00,None,None,1000,...,6.0,Chris Parry,Wanderson D'avila,55.0,7.09,7.51,8.78,9.0,8776,4.20


In [97]:
df.date.value_counts()

2021-04-03    1168
2020-10-17    1164
2021-05-29    1158
2020-11-03    1111
2021-05-22    1107
              ... 
2020-12-21     100
2021-03-22      95
2020-10-26      85
2021-03-15      62
2021-01-05      51
Name: date, Length: 351, dtype: int64

In [98]:
df.barrier.value_counts()

2     14336
1     14328
3     14310
4     14290
5     14237
6     13970
7     13484
8     12642
9     11245
10     9628
11     7820
12     5996
13     4426
14     3219
15     2265
16     1583
17     1042
18      678
19      283
20      167
21       45
22       20
23        8
24        5
Name: barrier, dtype: int64

In [99]:
df.race_distance.value_counts()

1200    15873
1400    11665
1200    10914
1000     9584
1600     8620
        ...  
1510        5
4000        5
2015        5
2206        5
1845        4
Name: race_distance, Length: 564, dtype: int64

In [100]:
# Functions
# __________________

def distance_group(distance):


    if distance is None:
        return("missing")
    elif distance < 1100:
        return("sprint")
    elif distance < 1400:
        return("mid_short")
    elif distance < 1800:
        return("mid_long")
    else:
        return("long")

def barrier_group(barrier):
    if barrier is None:
        return("missing")
    elif barrier < 4:
        return("inside")
    elif barrier < 9:
        return("mid_field")
    else:
        return("outside")


In [101]:
df['distance_group'] = pd.to_numeric(df.race_distance, errors = "coerce").apply(distance_group)
df['barrier_group'] = pd.to_numeric(df.barrier, errors = "coerce").apply(barrier_group)

In [102]:
# Apply Back Profit Less Commission

df['back_npl'] = np.where(df['place'] == 1, 0.95 * (df['bsp']-1), -1)
df['lay_npl'] = np.where(df['place'] == 1, -1 * (df['bsp']-1), 0.95)

# df.query('place == 1')[['bsp', 'place', 'back_npl']]


In [103]:
df.distance_group.value_counts()

mid_short    64998
mid_long     51038
sprint       23171
long         20820
Name: distance_group, dtype: int64

In [104]:
df.barrier_group.value_counts()

mid_field    68623
outside      48430
inside       42974
Name: barrier_group, dtype: int64

In [105]:
df_train = df.query('date < "2021-04-01"')
df_test = df.query('date >= "2021-04-01"')

In [106]:
'{} rows in the "training" set and {} rows in the "test" data'.format(df_train.shape[0], df_test.shape[0])

'119244 rows in the "training" set and 40783 rows in the "test" data'

# SIMPLE

Track / Barrier / Distance

In [107]:

# Simple

# Bet placement: BSP Back or Lay
# Strategy: Select a profitable track / distance / barrier group

trackDistanceBarrier = (
    df_train
    .assign(stake = 1)
    .assign(odds = lambda x: x['bsp'])
    .groupby(['track', 'race_distance', 'barrier_group'], as_index=False)
    .agg({'back_npl': 'sum', 'lay_npl': 'sum','stake': 'sum', 'odds': 'mean'})
)

trackDistanceBarrier



,track,race_distance,barrier_group,back_npl,lay_npl,stake,odds
0,Albany,1000,inside,11.2550,-11.95,2,15.450000
1,Albany,1000,mid_field,-5.0000,4.75,5,101.136000
2,Albany,1000,outside,-5.0000,4.75,5,88.374000
3,Albany,1100,inside,-3.0525,2.70,6,29.430000
4,Albany,1100,mid_field,-6.4040,5.92,9,37.483333
...,...,...,...,...,...,...,...
6325,York,1500,inside,1.8995,-2.41,6,41.195000
6326,York,1500,mid_field,-7.0000,6.65,7,32.472857
6327,York,1920,inside,-3.0000,2.85,3,21.883333
6328,York,1920,mid_field,-0.3520,-0.04,5,20.978000


In [108]:
def plStatisticalSignificance(df):

    x = (
        df
        .assign(pot=lambda x: x['npl'] / x['stake'])
        .assign(pot_abs=lambda x: abs(x['pot']))
        .assign(pot_sign=lambda x: np.where(x['pot']==x['pot_abs'], 1, -1))
        .assign(stake=lambda x: x['stake'].astype('float'))
        .assign(t_score=lambda x: (x['pot'] * np.sqrt(x['stake'])) / np.sqrt( (1 + x['pot']) * (x['odds'] - 1 - x['pot']) ) )
        .assign(p_value=lambda x: 2 * t.cdf(-abs(x['t_score']), x['stake']-1))
        .assign(p_value=lambda x: np.where(x['p_value']==0, 1, x['p_value']))
        .assign(p_value=lambda x: np.where(np.isnan(x['p_value']), 1, x["p_value"]))
        .drop(columns = ['pot', 'pot_abs', 'pot_sign', 't_score'])
    )

    return(x)


backStatTest = plStatisticalSignificance(
    trackDistanceBarrier.assign(npl=lambda x:x['back_npl'])
)

layStatTest = plStatisticalSignificance(
    trackDistanceBarrier.assign(npl=lambda x:x['lay_npl'])
)

/home/tmbish/.local/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning:

invalid value encountered in sqrt



In [109]:
backStatTest

,track,race_distance,barrier_group,back_npl,lay_npl,stake,odds,npl,p_value
0,Albany,1000,inside,11.2550,-11.95,2.0,15.450000,11.2550,0.487280
1,Albany,1000,mid_field,-5.0000,4.75,5.0,101.136000,-5.0000,1.000000
2,Albany,1000,outside,-5.0000,4.75,5.0,88.374000,-5.0000,1.000000
3,Albany,1100,inside,-3.0525,2.70,6.0,29.430000,-3.0525,0.754412
4,Albany,1100,mid_field,-6.4040,5.92,9.0,37.483333,-6.4040,0.532857
...,...,...,...,...,...,...,...,...,...
6325,York,1500,inside,1.8995,-2.41,6.0,41.195000,1.8995,0.918934
6326,York,1500,mid_field,-7.0000,6.65,7.0,32.472857,-7.0000,1.000000
6327,York,1920,inside,-3.0000,2.85,3.0,21.883333,-3.0000,1.000000
6328,York,1920,mid_field,-0.3520,-0.04,5.0,20.978000,-0.3520,0.972659


In [110]:
layStatTest

,track,race_distance,barrier_group,back_npl,lay_npl,stake,odds,npl,p_value
0,Albany,1000,inside,11.2550,-11.95,2.0,15.450000,-11.95,1.000000
1,Albany,1000,mid_field,-5.0000,4.75,5.0,101.136000,4.75,0.885995
2,Albany,1000,outside,-5.0000,4.75,5.0,88.374000,4.75,0.877954
3,Albany,1100,inside,-3.0525,2.70,6.0,29.430000,2.70,0.869397
4,Albany,1100,mid_field,-6.4040,5.92,9.0,37.483333,5.92,0.804366
...,...,...,...,...,...,...,...,...,...
6325,York,1500,inside,1.8995,-2.41,6.0,41.195000,-2.41,0.849635
6326,York,1500,mid_field,-7.0000,6.65,7.0,32.472857,6.65,0.755643
6327,York,1920,inside,-3.0000,2.85,3.0,21.883333,2.85,0.816546
6328,York,1920,mid_field,-0.3520,-0.04,5.0,20.978000,-0.04,0.996987


In [129]:
# Test Top 10 Lay Strategies
layTop5 = layStatTest.query('lay_npl>0').sort_values('p_value').head(5)
layTop5


,track,race_distance,barrier_group,back_npl,lay_npl,stake,odds,npl,p_value
188,Ascot,1000,inside,-83.6395,77.06,115.0,24.616870,77.06,0.248157
3619,Moonee Valley,1200,inside,-52.2195,48.81,64.0,17.725313,48.81,0.254399
6299,Yeppoon,1400,inside,-11.0000,10.45,11.0,6.022727,10.45,0.289686
959,Caulfield,1400,mid_field,-74.3150,67.45,114.0,24.828772,67.45,0.301137
1366,Darwin,1200,mid_field,-47.3980,43.94,64.0,19.354531,43.94,0.318178


In [130]:
trainSetTop10Lay = (
    df_train
    .merge(layTop5[['track', 'race_distance']])
    .assign(npl=lambda x: x['lay_npl'])
    .assign(stake=1)
)

testSetTop10Lay = (
    df_test
    .merge(layTop5[['track', 'race_distance']])
    .assign(npl=lambda x: x['lay_npl'])
    .assign(stake=1)
)

testSetTop10Lay

,date,track,country,race_type,race_number,market_id,start_time,weather,track_condition,race_distance,...,bsp,ltp,traded_vol,inplay_min_lay,distance_group,barrier_group,back_npl,lay_npl,npl,stake
0,2021-04-02,Ascot,AUS,R,6,1.181330624,2021-04-02 07:15:00,None,None,1000,...,11.08,11.5,4302,8.80,sprint,mid_field,-1.0000,0.95,0.95,1
1,2021-04-02,Ascot,AUS,R,6,1.181330624,2021-04-02 07:15:00,None,None,1000,...,5.41,5.4,22583,5.00,sprint,outside,-1.0000,0.95,0.95,1
2,2021-04-02,Ascot,AUS,R,6,1.181330624,2021-04-02 07:15:00,None,None,1000,...,4.73,4.6,33321,1.01,sprint,inside,3.5435,-3.73,-3.73,1
3,2021-04-02,Ascot,AUS,R,6,1.181330624,2021-04-02 07:15:00,None,None,1000,...,7.35,7.2,9892,7.00,sprint,mid_field,-1.0000,0.95,0.95,1
4,2021-04-02,Ascot,AUS,R,6,1.181330624,2021-04-02 07:15:00,None,None,1000,...,4.97,4.9,15590,4.80,sprint,outside,-1.0000,0.95,0.95,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,2021-06-26,Darwin,AUS,R,3,1.184733965,2021-06-26 05:37:00,None,None,1200,...,18.31,19.5,4171,19.50,mid_short,outside,-1.0000,0.95,0.95,1
220,2021-06-26,Darwin,AUS,R,3,1.184733965,2021-06-26 05:37:00,None,None,1200,...,42.00,40.0,1118,44.00,mid_short,mid_field,-1.0000,0.95,0.95,1
221,2021-06-26,Darwin,AUS,R,3,1.184733965,2021-06-26 05:37:00,None,None,1200,...,29.91,30.0,1090,9.60,mid_short,mid_field,-1.0000,0.95,0.95,1
222,2021-06-26,Darwin,AUS,R,3,1.184733965,2021-06-26 05:37:00,None,None,1200,...,6.60,6.4,14863,1.01,mid_short,outside,5.3200,-5.60,-5.60,1


In [131]:
def bet_eval_metrics(d, side = False):

    if side:
        metrics = (d
         .groupby('bet_side', as_index=False)
         .agg({"npl": "sum", "stake": "sum"})
         .assign(pot=lambda x: x['npl'] / x['stake'])
        )
    else:
        metrics = pd.DataFrame(d
         .agg({"npl": "sum", "stake": "sum"})
        ).transpose().assign(pot=lambda x: x['npl'] / x['stake'])

    return(metrics[metrics['stake'] != 0])



In [132]:
bet_eval_metrics(trainSetTop10Lay)


,npl,stake,pot
0,98.1,1047.0,0.093696


In [133]:
bet_eval_metrics(testSetTop10Lay)


,npl,stake,pot
0,18.44,224.0,0.082321


In [134]:
# Entire Period Cumulative PL

d = (
    df
    .merge(testSetTop10Lay[['track', 'race_distance']])
    .assign(npl=lambda x: x['lay_npl'])
    .assign(stake=1)
    .groupby(['date'], as_index=False)
    .agg({'npl': 'sum'})
)    

d['cNpl'] = d.npl.cumsum()

chart = px.line(d, x="date", y="cNpl", title='Cumulative Net Profit', template='simple_white')


In [135]:
chart

# Medium

Are Certain Jockeys or Trainers predictable shorteners or overbet?

- Predictable shorteners (hypothesise based on 5min WAP vs 30sec WAP)
- Overbet (look at PL stat sig of lay price 30s out)

In [143]:

# Jockey Shortening Rates

(
    df_train
    .assign(market_support=lambda x: x['wap_5m'] / x['wap_30s'])
    .assign(races=1)
    .groupby('jockey')
    .agg({'market_support': 'mean', 'races': 'count'})
    .query('races > 10')
    .sort_values('market_support', ascending = False)
)


,market_support,races
jockey,,
Scott Sheargold,1.133095,192
Lorelle Crow,1.056582,106
Chris Mc Carthy,1.051022,26
Anthony Darmanin,1.048931,142
James Winks,1.048893,12
...,...,...
Cyril Small,0.731661,16
A Kajiya,0.720798,21
J Luxe,0.710535,11


In [153]:
# Jockey 5 min PL Stat Sig

jockeys = (
    df_train
    .assign(stake = 1)
    .assign(odds = lambda x: x['wap_5m'])
    .assign(npl=lambda x: np.where(x['place'] == 1, 0.95 * (x['odds']-1), -1))
    .groupby(['jockey'], as_index=False)
    .agg({'odds': 'mean', 'stake': 'sum', 'npl': 'sum', })
)

jockeyStatTest = plStatisticalSignificance(jockeys)

jockeyStatTest.query('npl>0').sort_values('p_value')


,jockey,odds,stake,npl,p_value
134,Clayton Gallagher,28.126104,326.0,414.7215,0.002938
412,Michael Dee,25.597209,474.0,279.0425,0.038500
253,Jade Darose,97.873039,102.0,468.5560,0.043756
514,R Hutchings,26.314390,164.0,159.9365,0.073518
315,Justin P Stanley,10.245631,396.0,113.7680,0.093063
...,...,...,...,...,...
191,Emily Brown,64.667225,173.0,2.7540,0.979257
193,Emily Finnegan,32.998750,16.0,0.4970,0.983019
341,Krysten Swaffer,39.538153,158.0,0.1030,0.998949
508,Peter Wells,2.750000,1.0,1.6625,1.000000


In [158]:
bestJockey = (
    df_test
    .merge(jockeyStatTest.query('npl>0').sort_values('p_value').head(1)['jockey'])
    .assign(stake = 1)
    .assign(odds = lambda x: x['wap_5m'])
    .assign(npl=lambda x: np.where(x['place'] == 1, 0.95 * (x['odds']-1), -1))
)

bet_eval_metrics(bestJockey)



,npl,stake,pot
0,9.1775,148.0,0.06201


In [ ]:
# Best Jockey at BSP

bestJockeyBSP = (
    df_test
    .merge(jockeyStatTest.query('npl>0').sort_values('p_value').head(1)['jockey'])
    .assign(stake = 1)
    .assign(odds = lambda x: x['bsp'])
    .assign(npl=lambda x: np.where(x['place'] == 1, 0.95 * (x['odds']-1), -1))
)

bet_eval_metrics(bestJockey)

# Hard

Do certain jockeys or certain jockeys under certain track conditions present back to lay opportunities?